## 一、torch.nn类的Container
### 1.nn.Module
torch.nn.Module是PyTorch中所有神经网络模块的基类，任何自定义的网络结构、层等都需要继承这个类。其核心用途如下：

- **层次化结构**：模块可以包含子模块，从而构建复杂的网络结构（如将多个层组合成一个模块，然后再组合成整个网络）。
- **计算图支持**：定义前向传播（forward）方法后，PyTorch会自动跟踪计算图，支持自动求导。
- **参数管理**：模块可以跟踪其内部的所有nn.Parameter对象（通过parameters()方法访问），方便优化器的使用。
- **状态管理**：提供了将模型参数移动到GPU（.to(device)）、保存模型（state_dict）和加载模型（load_state_dict）的功能。

In [1]:
import torch.nn as nn
import torch.nn.functional as F

# 自定义神经网络模型Model，继承nn.Module的模版
class Model(nn.Module):

    # 定义初始化函数，神经网络结构
    def __init__(self):
        # 调用父类的初始化函数
        super(Model,self).__init__()
        # 实例变量，定义卷积层
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 20,kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 20,out_channels = 20,kernel_size = 5)

    # 前向传播函数
    def forward(self,x):
        # 两层卷积+非线性激活层（relu函数）
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x  # 返回结果

### 2.nn.Sequential
nn.Sequential是一个顺序容器，模块按照它们在序列中的顺序被依次执行，用于将多个模块按顺序组合起来。

前向传播时，输入数据依次通过序列中的每个模块，因此无需再另外定义前向传播函数。

模块的添加方式可以是通过传入一个有序字典（OrderedDict）或直接传入模块列表。适合快速构建简单的序列模型。

In [2]:
# 定义一个序列，依次插入神经网络的各个层
# 无需定义前向传播机制，自动按顺序运算
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

# 也可以传入有序字典OrderedDict，与上面的代码结果相同
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

### 3.nn.ModuleList
nn.ModuleList是一个用于保存子模块的列表，提供类似列表的操作（如添加append、索引访问__getitem__、迭代等），但没有类似nn.Sequential那样自动执行前向传播的功能。

In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # 动态创建层序列
        self.conv_layers = nn.ModuleList([
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        ])
        # 全连接层
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 10)
        )
    
    def forward(self, x):
        # 顺序执行所有卷积层
        for layer in self.conv_layers:
            x = layer(x)
        return self.classifier(x)

### 4.nn.ModuleDict
nn.ModuleDict本质是一个命名模块字典，具有类似字典的操作（键值对访问），与顺序无关，适用于多分支模型。

In [4]:
class ModuleDict(nn.Module):
    def __init__(self):
        super().__init__()
        # 按功能命名组织层
        self.features = nn.ModuleDict({
            'conv1': nn.Conv2d(3, 6, 5),
            'relu1': nn.ReLU(),
            'pool1': nn.MaxPool2d(2),
            'conv2': nn.Conv2d(6, 16, 5),
            'relu2': nn.ReLU(),
            'pool2': nn.MaxPool2d(2)
        })
        self.classifier = nn.ModuleDict({
            'flatten': nn.Flatten(),
            'fc1': nn.Linear(16 * 4 * 4, 120),
            'act': nn.ReLU(),
            'fc2': nn.Linear(120, 10)
        })
    
    def forward(self, x):
        # 按顺序调用特征层
        for layer in ['conv1', 'relu1', 'pool1', 'conv2', 'relu2', 'pool2']:
            x = self.features[layer](x)
        # 按顺序调用分类器
        for op in ['flatten', 'fc1', 'act', 'fc2']:
            x = self.classifier[op](x)
        return x

### 5.总结
- nn.Module是**基类**：是所有模型模块的基础，定义前向逻辑并管理参数/状态，适用任意复杂模型（分支/跳跃连接）。
- nn.Sequential是**顺序管道**：自动按添加顺序执行层，适用于线性结构模型，简化前向实现。
- nn.ModuleList是**动态容器**：存储层列表供迭代/索引访问，适用动态层数模型（如RNN堆叠），可append层但需手动前向。
- nn.ModuleDict是**命名字典**：按键名（字符串）组织层，适用多分支/条件选择结构（如多任务头），支持灵活访问与重组。

**内在联系**：
- 继承关系：Sequential/ModuleList/ModuleDict 都是 Module 的子类
- 协同工作：复杂模型中嵌套使用（如 Module 包含 ModuleDict 管理多头、Sequential管理模块等等）

## 二、卷积层
使用nn.Conv2d函数定义二维的卷积层（适用于图像），类的结构如下：

CLASS torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

- **in_channels**：输入通道数（如RGB图像为3，黑白图像为1）
- **out_channels**：输出通道数（输出的卷积核数量）
- **kernel_size**：卷积核的大小，为整数（正方形）或（height,weight）
- **stride**：步幅大小，整数或 (h_stride, w_stride)
- **padding**：填充数量，整数（上下左右填充数量相同）或（h_pad,w_pad）表示上下相同、左右相同。
- **dilation**：卷积核元素间的空洞间距（控制核膨胀）
- **groups**：输入/输出通道的分组数（1=标准卷积）
- **bias**：是否添加可学习的偏置项
- **padding_mode**：填充模式，默认为'zero'，除此之外还有'reflect'（镜像填充）、'replicate'（重复边缘值）和'circular'（循环填充）
- **device**：参数储存在CPU/GPU
- **dtype**：参数数据类型
## 三、池化层
池化层包括最大池化MaxPool2d和平均池化AvgPool2d，以MaxPool2d类为例，参数结构如下：

CLASS torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

- kernel_size：滑动窗口（池化核）尺寸，整数（方形核）或元组 (height, width)
- stride：窗口移动步长
- padding：填充个数
- dilation：池化核元素间隔（扩大感受野）
- return_indices：是否返回最大值位置索引，默认False
- ceil_mode：尺寸计算方式，默认False，向下取整

## 四、数据获取与加载
### 1. Dataset
Dataset旨在提供一种方式，获取数据集及其对应的真实label值，以便为模型提供合适的输入格式。Dataset是一个类，可以通过dir()访问类的实例方法，通过help()查看类的定义和使用方法。

### 2.DataLoader
Dataset负责指定一个数据集，而DataLoader则负责将数据集读取、并加载到神经网络内。
DataLoader类包括的参数有：

CLASS torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=None, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None, generator=None, *, prefetch_factor=None, persistent_workers=False, pin_memory_device='', in_order=True)

- **dataset**：要加载的数据集对象（Dataset子类实例）
- **batch_size**：每个批次的样本数，神经网络常见的超参数之一（默认为1）
- **shuffle**：是否在每个epoch前打乱数据顺序，一般设为True（默认为False）
- **sampler**：自定义采样对象，与shuffle互斥
- **batch_sampler**：返回整个批次索引的采样器（替代batch_size+sampler）
- **number_workers**：数据加载子进程数（0=主进程加载，>0=并行加载），默认为0
- **drop_last**：是否丢弃最后不足batch_size的剩余样本（默认False保留）
- prefetch_factor：每个worker预取的批次数量（默认=2）
- persistent_workers：是否保持worker进程存活（减少进程重建开销）
- pin_memory：是否将数据复制到锁页内存（加速GPU传输）
- pin_memory_device​​：指定锁页内存设备（如‘cuda’）
- collate_fn​​：自定义批次样本组合函数（处理不规则数据）
- worker_init_fn​​：worker进程初始化函数（设置随机种子等）
- timeout​​：数据加载超时时间（秒，默认0=无限等待）
- multiprocessing_context​​：多进程启动方式（‘spawn’/‘fork’）
- generator​​：指定随机数生成器（控制shuffle的随机性）
- in_order​​：分布式场景下是否按顺序返回数据（默认True避免性能开销）

In [17]:
# torch.utils.data包含了处理数据的一般工具包
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms


# 定义存储位置及批次大小
data_dir = '/kaggle/working'
batch_size = 256

# 加载CIFAR-10数据集
train_dataset = torchvision.datasets.CIFAR10(
    root=data_dir,
    train=True,
    download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root=data_dir,
    train=False,
    download=True
)


# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 170M/170M [00:13<00:00, 13.0MB/s] 


## 五、Torchvision的使用
torchvision是Pytorch应用于计算机视觉领域的**扩展库**，主要提供：

- 流行的数据集：如MNIST, CIFAR10, ImageNet，可以直接通过torchvision库导入数据；
- 常见的模型架构：如AlexNet, ResNet, VGG，以及预训练权重；
- 图像变换和增强操作：用于数据预处理和数据增强；
- 工具函数（如绘制边界框、网格保存图像等）
### 1.torchvision导入数据
可以通过dir()查看内置数据集：dir(torchvision.datasets)。以CIFAR10为例：

CLASS torchvision.datasets.CIFAR10(root: Union[str, Path], train: bool = True, transform: Optional[Callable] = None, target_transform: Optional[Callable] = None, download: bool = False)

- **root**：数据存储路径
- **train**：是否为训练集
- **transform**：数据预处理方法，默认为None
- **target_transform**：标签预处理变换，默认为None
- **download**：是否下载数据集，默认为False，同一路径下不会重复下载
### 2.transforms.Compose数据预处理
torchvision库的transforms.Compose专门用于数据预处理和数据增强，常用的方法包括：

张量转换及标准化：
- **transforms.ToTensor()**：PIL→Tensor，将图片转换为张量格式
- **transforms.Normalize()**：张量标准化，输入各通道mean、std，列表格式
- **transforms.Resize()**：尺寸调整，设定size（int/(h,w))
- **transforms.CenterCrop()**：中心裁剪，设定裁剪size
- **transforms.RandomCrop()**：随机裁剪，设定裁剪大小size(int/(h,w))、填充padding
- **transforms.ColorJitter()**：亮度brightness、对比度contrast、饱和度saturation、色相偏移hue
- **transforms.GausssianBlur()**：随机选定高斯模糊核模糊图像，参数包括kernel_size、sigma（模糊程度的方差）
- **transforms.RandomHorizontalFlip()**：随机水平翻转
- **transforms.RandomRotation()**：随机角度的旋转

In [19]:
import torchvision
import torchvision.transforms as transforms

# 数据处理
transform = transforms.Compose([
    transforms.Resize(256),                     # 尺寸调整
    transforms.RandomCrop(224),                 # 随机裁剪
    transforms.RandomHorizontalFlip(p=0.5),     # 随机水平翻转
    transforms.ColorJitter(
        brightness=0.2,                        # 亮度调整幅度
        contrast=0.2,                          # 对比度调整幅度
        saturation=0.2,                        # 饱和度调整幅度
        hue=0.02 ),                              # 色相偏移范围(-0.5,0.5)
    transforms.ToTensor(),                     # 转为Tensor
    transforms.Normalize(                      # 归一化
        mean=[0.485, 0.456, 0.406],           # 通道均值（分别计算三个通道的均值）
        std=[0.229, 0.224, 0.225]             # 通道标准差（分别计算三个通道的标准差）
    )])

# 加载CIFAR-10数据集
data_dir = "/kaggle/working"
train_dataset = torchvision.datasets.CIFAR10(
    root=data_dir,
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root=data_dir,
    train=False,
    download=True,
    transform=transform
)

In [20]:
import torch
from torch.utils.data import Dataset
help(Dataset)

Help on class Dataset in module torch.utils.data.dataset:

class Dataset(typing.Generic)
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`. Subclasses could also
 |  optionally implement :meth:`__getitems__`, for speedup batched samples
 |  loading. This method accepts list of indices of samples of batch and returns
 |  list of samples.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs an index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral ind